# Дифференциально экспрессированные гены

## Описание используемых данных
Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

## Практическая значимость изменения
Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:

$$\begin{equation*}
F_c(C, T) = 
 \begin{cases}
   \frac{T}{C}, &\text{$T > C$}\\
   \frac{-C}{T}, &\text{$T < C$}
 \end{cases}
\end{equation*}$$

где $C, T$ — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

## Инструкции к решению задачи
Задание состоит из трёх частей. Если не сказано обратное, то уровень значимости нужно принять равным 0.05.  
Данные находятся в файле **gene_high_throughput_sequencing.csv**

### Часть 1: применение t-критерия Стьюдента
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

1. для групп normal (control) и early neoplasia (treatment)
2. для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
%matplotlib inline 

In [2]:
data = pd.read_csv('datasets/gene_high_throughput_sequencing.csv')
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [3]:
data.shape

(72, 15750)

In [4]:
data.Diagnosis.value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [5]:
normal = data[data.Diagnosis == 'normal'].iloc[:, 2:]
early_neoplasia = data[data.Diagnosis == 'early neoplasia'].iloc[:, 2:]
cancer = data[data.Diagnosis == 'cancer'].iloc[:, 2:]

In [6]:
print(normal.shape, early_neoplasia.shape, cancer.shape)

(24, 15748) (25, 15748) (23, 15748)


In [7]:
ttest1 = [ttest_ind(normal[column], early_neoplasia[column], equal_var=False).pvalue for column in data.columns[2:]]

In [8]:
n_ttest1 = len([p_value for p_value in ttest1 if p_value < 0.05])
n_ttest1

1575

In [9]:
ttest2 = [ttest_ind(early_neoplasia[column], cancer[column], equal_var=False).pvalue for column in data.columns[2:]]

In [10]:
n_ttest2 = len([p_value for p_value in ttest2 if p_value < 0.05])
n_ttest2

3490

In [11]:
with open('task1_1.txt', 'w') as f1_1, open('task1_2.txt', 'w') as f1_2:
    f1_1.write(str(n_ttest1))
    f1_2.write(str(n_ttest2))

### Часть 2: поправка методом Холма
Для этой части задания вам понадобится модуль multitest из statsmodels.

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5

Обратите внимание, что

* применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.
* при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

In [14]:
import statsmodels.stats.multitest as smm

In [56]:
def check_fold_change(C, T):
    fold_changes = [t/c if t > c else -c/t for c, t in zip(C, T)]
    return len([fc for fc in fold_changes if abs(fc) > 1.5])
    

In [57]:
holm_multitests1 = smm.multipletests(ttest1, alpha=0.05/2, method='holm')
rejected1 = holm_multitests1[0]

In [58]:
controls1 = normal.iloc[:, rejected1].mean(axis=0)
controls1

PCSK4     14.983424
EEF1A2    19.373612
dtype: float64

In [59]:
treatments1 = early_neoplasia.iloc[:, rejected1].mean(axis=0)
treatments1

PCSK4     22.621756
EEF1A2    38.260320
dtype: float64

In [60]:
ans2_1 = check_fold_change(controls1, treatments1)
ans2_1

2

In [61]:
holm_multitests2 = smm.multipletests(ttest2, alpha=0.05/2, method='holm')
rejected2 = holm_multitests2[0]

In [62]:
controls2 = early_neoplasia.iloc[:, rejected2].mean(axis=0)
controls2

GABRD             7.224275
EXTL1            23.234551
CD164L2          23.734858
NES              93.810978
FMO2             11.619290
GLT25D2          13.853744
ITIH5            23.550369
HPSE2             9.975886
SYT8             79.807508
TNNI2            58.744208
ALX4              7.432551
PDGFD            20.010590
CRYAB            36.731940
TRIM29           75.542892
WIF1             12.856368
TESC             38.740644
KLF5             12.814674
SPRY2            15.142730
NDRG2           119.553639
CMTM5             6.586257
SSTR1             5.248819
FBLN5            20.530677
LOC100507043      3.878519
SOX8             17.196034
HS3ST6            9.254571
DPEP1             6.171166
STAC2            72.407037
CD300LG          10.359332
CACNA1G          17.936385
LAMA3            19.044810
                   ...    
D4S234E          10.328128
ENPP6             3.966548
SEMA5A           17.431515
ALDH7A1          13.544707
SLC27A6           5.751107
PSD2              5.155417
N

In [63]:
treatments2 = cancer.iloc[:, rejected2].mean(axis=0)
treatments2

GABRD           13.320553
EXTL1            9.749268
CD164L2         11.882991
NES             55.397100
FMO2             6.470836
GLT25D2          6.536257
ITIH5           14.447875
HPSE2            4.073066
SYT8            26.308148
TNNI2           21.026743
ALX4             3.496135
PDGFD           10.326469
CRYAB           21.937459
TRIM29          37.677675
WIF1             2.945439
TESC            16.673402
KLF5             7.334227
SPRY2            7.402410
NDRG2           72.565646
CMTM5            2.406401
SSTR1            2.452194
FBLN5           13.255595
LOC100507043     1.505345
SOX8             9.273945
HS3ST6           3.185174
DPEP1           14.124519
STAC2           27.058619
CD300LG          4.689680
CACNA1G          8.957433
LAMA3            8.843209
                  ...    
D4S234E          5.721148
ENPP6            1.826406
SEMA5A           8.766100
ALDH7A1          8.205916
SLC27A6          2.085935
PSD2             2.165149
NRG2             7.569859
IL17B       

In [64]:
ans2_2 = check_fold_change(controls2, treatments2)
ans2_2

77

In [66]:
with open('task2_1.txt', 'w') as f2_1, open('task2_2.txt', 'w') as f2_2:
    f2_1.write(str(ans2_1))
    f2_2.write(str(ans2_2))

### Часть 3: поправка методом Бенджамини-Хохберга
Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от $H_0$, когда они есть, и будут чаще отклонять $H_0$ когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [78]:
multitests_bh1 = smm.multipletests(ttest1, alpha=0.05/2, method='fdr_bh')
rejected_bh1 = multitests_bh1[0]

In [79]:
controls_bh1 = normal.iloc[:, rejected_bh1].mean(axis=0)
controls_bh1

TMEM63C    18.037315
PCSK4      14.983424
CACNG8      3.333050
EEF1A2     19.373612
dtype: float64

In [80]:
treatments_bh1 = early_neoplasia.iloc[:, rejected_bh1].mean(axis=0)
treatments_bh1

TMEM63C    30.010954
PCSK4      22.621756
CACNG8      5.849340
EEF1A2     38.260320
dtype: float64

In [81]:
ans3_1 = check_fold_change(controls_bh1, treatments_bh1)
ans3_1

4

In [82]:
multitests_bh2 = smm.multipletests(ttest2, alpha=0.05/2, method='fdr_bh')
rejected_bh2 = multitests_bh2[0]

In [83]:
controls_bh2 = early_neoplasia.iloc[:, rejected_bh2].mean(axis=0)
controls_bh2

ISG15          37.296084
FAM132A        19.057449
GNB1           28.791099
GABRD           7.224275
PLCH2          26.903924
C1orf93        11.044611
AJAP1           4.861378
CA6             2.985346
SLC2A5          4.741555
MASP2           5.007323
FBXO6          10.459531
LRRC38          5.881019
EFHD2          37.960590
MFAP2           2.555026
ALPL           27.070752
WNT4           36.882703
EXTL1          23.234551
SH3BGRL3      106.965250
AIM1L           3.627885
TRNP1           8.951046
CD164L2        23.734858
LAPTM5         66.993366
CLSPN           2.820314
RSPO1           7.570254
CLDN19          9.369520
CDC20           7.636862
KIF2C           4.057512
FAM159A         6.634695
ECHDC2         18.125786
WLS            15.903770
                 ...    
WAS             9.433534
SLC7A3          9.553186
ITGB1BP2        2.216399
CXCR3           7.684186
RGAG4          11.751430
CITED1         22.352744
ITM2A          16.309607
ZNF711          9.302049
GPRASP1         9.067673


In [84]:
treatments_bh2 = cancer.iloc[:, rejected_bh2].mean(axis=0)
treatments_bh2

ISG15          91.706112
FAM132A        14.330161
GNB1           33.628745
GABRD          13.320553
PLCH2          19.128238
C1orf93        14.955557
AJAP1           3.100736
CA6             1.718914
SLC2A5          6.932564
MASP2           2.861000
FBXO6          14.956427
LRRC38          2.216106
EFHD2          45.443819
MFAP2           4.863978
ALPL           18.442687
WNT4           16.869454
EXTL1           9.749268
SH3BGRL3      140.474137
AIM1L           7.177703
TRNP1           5.507176
CD164L2        11.882991
LAPTM5         99.825864
CLSPN           5.881522
RSPO1           4.483246
CLDN19          4.408661
CDC20          15.631632
KIF2C           6.917925
FAM159A        11.041758
ECHDC2         12.948872
WLS            10.518910
                 ...    
WAS            13.404642
SLC7A3          2.962692
ITGB1BP2        3.374989
CXCR3          14.636917
RGAG4           8.904966
CITED1         11.954509
ITM2A           9.147393
ZNF711          5.164723
GPRASP1         6.158201


In [85]:
ans3_2 = check_fold_change(controls_bh2, treatments_bh2)
ans3_2

524

In [86]:
with open('task3_1.txt', 'w') as f3_1, open('task3_2.txt', 'w') as f3_2:
    f3_1.write(str(ans3_1))
    f3_2.write(str(ans3_2))